# Projeto Final de Mineração de Dados

## Alunos: Fabrício Moreno da Silva & Ynnayron Juan Lopes da Silva

## Professor: Paulo Ribeiro Lins Júnior

---

# Objetivo do Projeto

### O objetivo deste projeto é utilizar inteligência artificial (IA) para prever a taxa de churn em uma empresa, identificando padrões e fatores que possam levar à perda de clientes. Com base nessas previsões, o projeto visa ajudar a propor estratégias e soluções eficazes para reduzir o churn, ajudando a empresa a reter seus clientes e otimizar seus processos de fidelização.

---

## Seguindo os passos para chegar ao resultado

---

# 1. Importando as bibliotecas necessárias

In [2]:
import pandas as pd
import numpy as np
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV


---

### Seguindo adiante, fazemos a importação do nosso dataset

In [4]:
df_telecom = pd.read_csv('churn_projeto_final.csv')

In [ ]:
df_telecom

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,AZ,192,415,414-4276,no,yes,36,156.2,77,26.55,...,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
3329,WV,68,415,370-3271,no,no,0,231.1,57,39.29,...,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
3330,RI,28,510,328-8230,no,no,0,180.8,109,30.74,...,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
3331,CT,184,510,364-6381,yes,no,0,213.8,105,36.35,...,84,13.57,139.2,137,6.26,5.0,10,1.35,2,False


---

### Depois de importar nossos dados, vamos partir para as funções de processamento e limpeza de dados

#### Nossa primeira função tem o intuíto de procurar dados ausentes no nosso dataset

In [ ]:
def check_missing_data(df):
    missing_data = df.isnull().sum()
    print("Dados ausentes em cada coluna:\n", missing_data)

    return missing_data

#### Em seguida, vamos fazer um recategorização de alguns dados

In [ ]:
def conversao_coluna_categorica(df, columns, encoding_type='label'):
    if encoding_type == 'label':
        for col in columns:
            df[col] = df[col].map({'yes': 1, 'no': 0})
    elif encoding_type == 'onehot':
        df = pd.get_dummies(df, columns=columns, drop_first=True)
    else:
        raise ValueError("Tipo de codificação inválido. Use 'label' ou 'onehot'.")

    return df


#### Por fim, temos uma função que vai padronizar nossos dados

In [ ]:
from sklearn.preprocessing import StandardScaler

def escalonar_variaveis(df, numeric_columns):
    scaler = StandardScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    return df


---

### Seguindo adiante, vamos começar com o treinamento dos nossos modelos. Para isso, fazemos a separação dos nossos dados, entre dados de teste e dados de treino

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(df, target_column, test_size=0.3, random_state=42):
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test


In [ ]:
def drop_columns(df, columns_to_drop):
    df_cleaned = df.drop(columns=columns_to_drop, axis=1)
    return df_cleaned


In [ ]:
from imblearn.over_sampling import SMOTE

def balance_data(X_train, y_train):
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    return X_train_balanced, y_train_balanced


---

### Adiante, vamos utilizar nossas funções para realizar seus propósitos, e nos ajudar a manter um resultado satisfatório

In [ ]:
# Verificar dados ausentes
check_missing_data(df_telecom)

# Codificar variáveis categóricas
df_encoded = conversao_coluna_categorica(df_telecom, columns=['international plan', 'voice mail plan'], encoding_type='label')

# Escalonar variáveis numéricas
numeric_columns = ['account length', 'total day minutes', 'total day calls', 'total day charge',
                   'total eve minutes', 'total eve calls', 'total eve charge',
                   'total night minutes', 'total night calls', 'total night charge',
                   'total intl minutes', 'total intl calls', 'total intl charge',
                   'customer service calls']
df_scaled = escalonar_variaveis(df_encoded, numeric_columns)
columns_to_drop = ['state','phone number']
df_cleaned = drop_columns(df_scaled, columns_to_drop)
df_cleaned= df_cleaned.fillna(0)

X_train, X_test, y_train, y_test = split_data(df_cleaned, target_column='churn')

# Balancear os dados de treino
X_train_balanced, y_train_balanced = balance_data(X_train, y_train)


Dados ausentes em cada coluna:
 state                     0
account length            0
area code                 0
phone number              0
international plan        0
voice mail plan           0
number vmail messages     0
total day minutes         0
total day calls           0
total day charge          0
total eve minutes         0
total eve calls           0
total eve charge          0
total night minutes       0
total night calls         0
total night charge        0
total intl minutes        0
total intl calls          0
total intl charge         0
customer service calls    0
churn                     0
dtype: int64


### Como pôde ser visto, nossas colunas estão limpas, nos permitindo seguir adiante sem interferências externas

---

# 2. Utilização de Diferentes Treinamentos

---

## Agora chegamos ao ponto alto de todo o processo. Agora, vamos treinar nossos modelos, são eles:
- Regressão Logística
- Random Forest
- SVM
- Naive Bayes
- Rede Neural

In [ ]:
# Função para amostragem de um conjunto de dados menor
def sample_data(X, y, sample_size=0.3):
    return train_test_split(X, y, test_size=sample_size, random_state=42)

# Função para treinar múltiplos modelos rapidamente
def train_models(X_train, y_train, models):
    results = {}
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for name, model in models.items():
        print(f"Treinando o modelo {name}...")
        scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy')
        results[name] = {'mean_accuracy': scores.mean(), 'std_accuracy': scores.std()}

    return results

# Função para analisar as variáveis mais significantes
def analyze_feature_importance(model, X_train):
    if hasattr(model, 'coef_'):
        importance = model.coef_[0]
    elif hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
    else:
        print("O modelo não suporta análise de importância das características.")
        return

    feature_importance = sorted(zip(X_train.columns, importance), key=lambda x: x[1], reverse=True)
    for feature, importance in feature_importance:
        print(f"{feature}: {importance:.4f}")

# Função para analisar os tipos de erros cometidos pelos modelos
def analyze_errors(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Função para configurar e rodar todo o pipeline de modelos
def run_experiment(X, y):
    # Amostragem de dados
    X_train, X_test, y_train, y_test = sample_data(X, y, sample_size=0.3)

    # Dicionário de modelos a serem testados
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(),
        'SVM': SVC(),
        'Naive Bayes': GaussianNB(),
        'Rede Neural': MLPClassifier(max_iter=500)
    }

    # Treinamento dos modelos
    model_results = train_models(X_train, y_train, models)
    print("\nResultados dos Modelos:")
    for name, result in model_results.items():
        print(f"{name}: Acurácia Média = {result['mean_accuracy']:.4f}, Desvio Padrão = {result['std_accuracy']:.4f}")

# Exemplo de uso
# X, y representam as características e os rótulos, que devem ser preparados previamente
# run_experiment(X, y)


---

### Depois da implementação dos nossos modelos, chegou o momento de treiná-los e ver o desempenho de cada um, que será mostrado abaixo

In [ ]:
run_experiment(X_train_balanced, y_train_balanced)

Treinando o modelo Logistic Regression...
Treinando o modelo Random Forest...
Treinando o modelo SVM...
Treinando o modelo Naive Bayes...
Treinando o modelo Rede Neural...

Resultados dos Modelos:
Logistic Regression: Acurácia Média = 0.7194, Desvio Padrão = 0.0188
Random Forest: Acurácia Média = 0.9319, Desvio Padrão = 0.0089
SVM: Acurácia Média = 0.5369, Desvio Padrão = 0.0088
Naive Bayes: Acurácia Média = 0.7348, Desvio Padrão = 0.0150
Rede Neural: Acurácia Média = 0.7376, Desvio Padrão = 0.0435


---

### Feito isso, vamos selecionar apenas os 3 melhores modelos, afinal, buscamos apenas o melhor resultado, e isso vai diminuir a nossa amostragem, e ver o comportamento de cada um de maneira mais detalhada

In [ ]:
    # Escolher os 3  melhors modelos para análise mais detalhada
modelo1 = RandomForestClassifier()
modelo1.fit(X_train, y_train)

    # Analisar importância das variáveis
print("\nImportância das Variáveis para Random Forest:")
analyze_feature_importance(modelo1, X_train)

    # Analisar erros do modelo
print("\nAnálise de Erros:")
analyze_errors(modelo1, X_test, y_test)


Importância das Variáveis para Random Forest:
total day minutes: 0.1487
customer service calls: 0.1176
total day charge: 0.1122
international plan: 0.0881
total eve charge: 0.0659
total eve minutes: 0.0655
total intl minutes: 0.0470
total intl calls: 0.0459
total intl charge: 0.0435
total night charge: 0.0404
total night minutes: 0.0388
total night calls: 0.0340
total day calls: 0.0320
account length: 0.0318
total eve calls: 0.0302
number vmail messages: 0.0280
voice mail plan: 0.0217
area code: 0.0087

Análise de Erros:
Confusion Matrix:
 [[852   5]
 [ 46  97]]

Classification Report:
               precision    recall  f1-score   support

       False       0.95      0.99      0.97       857
        True       0.95      0.68      0.79       143

    accuracy                           0.95      1000
   macro avg       0.95      0.84      0.88      1000
weighted avg       0.95      0.95      0.95      1000



### Com a análise feita, percebemos que a coluna "total day minutes", que .mostra a quantidade de minutos utilizados por dia da operadora, é a variável que mais importa pra gente.
### É de extrema importância entender isso, afinal, buscamos ter o menor número futuro de churns possíveis para a empresa, e sabendo do que mais afeta os clientes, é de suma importância para que possamos alcançar nossos objetivos com o maior êxito possível

In [ ]:
    # Escolher os 3  melhors modelos para análise mais detalhada
modelo2 = GaussianNB()
modelo2.fit(X_train, y_train)

    # Analisar importância das variáveis
print("\nImportância das Variáveis para Naive Bayes:")
analyze_feature_importance(modelo2, X_train)

    # Analisar erros do modelo
print("\nAnálise de Erros:")
analyze_errors(modelo2, X_test, y_test)


Importância das Variáveis para Naive Bayes:
O modelo não suporta análise de importância das características.

Análise de Erros:
Confusion Matrix:
 [[783  74]
 [ 77  66]]

Classification Report:
               precision    recall  f1-score   support

       False       0.91      0.91      0.91       857
        True       0.47      0.46      0.47       143

    accuracy                           0.85      1000
   macro avg       0.69      0.69      0.69      1000
weighted avg       0.85      0.85      0.85      1000



In [ ]:
    # Escolher os 3  melhors modelos para análise mais detalhada
modelo3 = MLPClassifier(max_iter=500)
modelo3.fit(X_train, y_train)

    # Analisar importância das variáveis
print("\nImportância das Variáveis para Redes Neurais:")
analyze_feature_importance(modelo3, X_train)

    # Analisar erros do modelo
print("\nAnálise de Erros:")
analyze_errors(modelo3, X_test, y_test)


Importância das Variáveis para Redes Neurais:
O modelo não suporta análise de importância das características.

Análise de Erros:
Confusion Matrix:
 [[849   8]
 [134   9]]

Classification Report:
               precision    recall  f1-score   support

       False       0.86      0.99      0.92       857
        True       0.53      0.06      0.11       143

    accuracy                           0.86      1000
   macro avg       0.70      0.53      0.52      1000
weighted avg       0.82      0.86      0.81      1000



# 3. COM ISSO O MODELO ESCOLHIDO PARA PRODUÇÃO SERÁ O RANDOM FOREST

---

### Depois de uma análise mais detalhada, podemos ver que o modelo Random Forest pode nos ajudar a alcançar os nossos objetivos com mais precisão. Então vamos em frente com ele, e finalmente utilizá-lo com mais dados, colocando-o realmente à prova

In [ ]:
# Função para realizar ajuste fino de hiperparâmetros
def hyperparameter_tuning(X_train, y_train, model, param_distributions, n_iter=50, cv=5):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=n_iter, cv=cv, scoring='accuracy', n_jobs=-1)
    random_search.fit(X_train, y_train)
    return random_search.best_params_, random_search.best_score_

# Definindo o modelo e os parâmetros a serem explorados
rf = RandomForestClassifier()
param_distributions_rf = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

# Ajustando o Random Forest
best_params_rf, best_score_rf = hyperparameter_tuning(X_train, y_train, rf, param_distributions_rf)
print("Melhores parâmetros Random Forest:", best_params_rf)
print("Melhor score Random Forest:", best_score_rf)

Melhores parâmetros Random Forest: {'max_depth': 46, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 295}
Melhor score Random Forest: 0.9541369898263963


### Como é possível ver acima, o modelo nos rendeu uma acurácia de incríveis 95% de precisão, um resultado muito mais do que satisfatório!

In [ ]:
modelo_final = RandomForestClassifier(max_depth= 46, min_samples_leaf = 1, min_samples_split= 2, n_estimators= 295)

In [ ]:
modelo_final.fit(X_train, y_train)

RandomForestClassifier(max_depth=46, n_estimators=295)

In [ ]:
def testar_modelo(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Acurácia no conjunto de teste: {accuracy:.4f}")
    print("Relatório de Classificação:\n", classification_report(y_test, y_pred))
    print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))

# Avaliação final do ensemble no conjunto de teste
testar_modelo(modelo_final, X_test, y_test)

Acurácia no conjunto de teste: 0.9520
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.95      1.00      0.97       857
        True       0.96      0.69      0.80       143

    accuracy                           0.95      1000
   macro avg       0.96      0.84      0.89      1000
weighted avg       0.95      0.95      0.95      1000

Matriz de Confusão:
 [[853   4]
 [ 44  99]]


# 4. Considerações Finais

---

### Em conclusão, o projeto de Mineração de Dados alcançou resultados significativos ao aplicar diferentes modelos preditivos com o objetivo de reduzir o número de futuros churns de clientes. Dentre os modelos testados, o Random Forest se destacou como o mais eficaz, atingindo uma acurácia média de 95%. Isso demonstra a sua capacidade robusta de classificação e identificação de padrões associados ao churn. Além disso, a análise de importância das variáveis revelou que a coluna "total day minutes" foi a mais relevante para o desempenho do modelo, indicando uma forte correlação entre o tempo total de minutos utilizados pelos clientes durante o dia e a probabilidade de churn. Esses resultados oferecem insights valiosos para a empresa desenvolver estratégias preventivas mais eficazes, focando nos comportamentos que mais influenciam a perda de clientes.






